In [5]:
from glob import glob
import json
import kagglehub
import numpy as np
import os
import pandas as pd
import requests
import shutil

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Download data from AIcrowd:
```bash
aicrowd login
aicrowd dataset download --challenge task-1-next-product-recommendation
```


In [14]:
for csv_file in glob(r'..\data\amazon_kdd\*.csv'):
    df = pd.read_csv(csv_file)
    print(csv_file)
    print(df.shape)
    display(df.sample(5))
    print()

..\data\amazon_kdd\products_train.csv
(1551057, 11)


,id,locale,title,price,brand,color,size,model,material,author,desc
861781,B083V1P4XV,JP,エリエール トイレットペーパー フラワープリント 25m×72ロール(12ロール×6パック)...,2978.00,エリエール,NaN,12ロール (x 6),NaN,木材パルプ,NaN,内容量:72ロール
47341,B08BFQ52PR,DE,"NIMASO USB C zu 3.5mm Klinke,USB C Aux Adapter...",9.99,NIMASO,Grau,NaN,NAD22A441,NaN,NaN,🎧『 Gute Haltbarkeit』Der Aux Adapter USB C aus ...
1510216,B0B8N2T8VK,IT,"Opret Bottiglie da Viaggio 100ml in Silicone, ...",12.99,Opret,"Blu, Rosa, Verde, Giallo","4 pezzi, 100ml",NaN,Silicone,NaN,【Approvato dalla TSA】 Approvato dalla compagni...
306684,B08YJLQXJR,DE,"Primst Katzentunnel großer Katzen Tunnel, Falt...",24.59,Primst,Grey 51x12inch,NaN,NaN,Wildleder,NaN,"Geniales Design: Der Tunnel ist faltbar, mit L..."
95943,B07LCCWBPW,DE,Mardingtop 28L Rucksack Taktischer Trekkingruc...,46.99,Mardingtop,28l Neu Schwarz,"18,1 x 10,8 x 8,7 Zoll",M6290,Polyester,NaN,☞ AUSREICHENDE KAPAZITAT. Im Inneren des Haupt...



..\data\amazon_kdd\sessions_test_task1.csv
(316972, 2)


,prev_items,locale
8006,['B07C8Q5FJ5' 'B09VXWMZDG' 'B08F21NLMN' 'B0B7K...,DE
237328,['B09HL2Y87M' 'B0B18YT6FF' 'B009IK5Q1Q'],UK
261090,['B09MCK3RQW' 'B09MCK2KK7' 'B09MCK7PXS' 'B09MC...,UK
210086,['B01N6BLHCN' 'B00B0KFO8W' 'B08H133S2L'],UK
307897,['B000TARAWM' 'B000MBVYJS' 'B002NKS30E' 'B00K6...,UK



..\data\amazon_kdd\sessions_test_task1_phase1.csv
(316971, 2)


...

,prev_items,locale
31009,['B09GM5NG9Z' 'B09KG9SMBV' 'B07V4SJF94'],IT
2475,['B09QQG85HM' 'B09K7TDY1H' 'B0998ZJCKF' 'B099K...,ES
14588,['B091MVWGMZ' 'B091M89RV4'],DE
39568,['B01I13QR22' 'B01I13R86Q' 'B01I13Q9CU' 'B01I1...,JP
15352,['B07WDRRF4T' 'B07WDRRF4T'],DE



..\data\amazon_kdd\sessions_test_task3_phase1.csv
(56421, 2)


,prev_items,locale
14642,['B07HD1LS78' 'B07HD1LS78' 'B07HD1LS78'],DE
15794,['B08B11873H' 'B08B11873H'],DE
45722,['B09S393TTF' 'B09S3G6QVL'],JP
39561,['B09PCC9ZW3' 'B09SG193VS' 'B09SG22CKN' 'B09L7...,JP
30993,['B01N11IJR0' 'B08YXMMB1J'],IT



..\data\amazon_kdd\sessions_train.csv
(3606249, 3)


,prev_items,next_item,locale
2879185,['B09B283Z2P' 'B09B2B5CZQ' 'B09FDFYKZ5' 'B09B2...,B091CJC21Z,UK
2252902,['B07DLYQCGT' 'B07GTDZMZP'],B08LHGWF83,UK
3334756,['B01EHSNIW2' 'B00129AI6O'],B08XZLYCZ5,ES
2108213,['B08P1XW7LF' 'B084SCK678' 'B08P1XW7LF' 'B084S...,B015FEY5OM,UK
556349,['B004JK7CC6' 'B0054DWTES'],B000H8B180,DE


In [3]:
# Let's just focus on task 1, the sessions training data, and the products training data:

products_df = pd.read_csv(r'..\data\amazon_kdd\products_train.csv')
sessions_df = pd.read_csv(r'..\data\amazon_kdd\sessions_train.csv')

In [4]:
# Focus on UK locale for now
# Maybe chase a multilingual embedding model soon and support everything
products_df = products_df[products_df['locale'] == 'UK'].reset_index(drop=True)
sessions_df = sessions_df[sessions_df['locale'] == 'UK'].reset_index(drop=True)

print(products_df.shape)
display(products_df.sample(5))

print(sessions_df.shape)
display(sessions_df.sample(5))

(500180, 11)


,id,locale,title,price,brand,color,size,model,material,author,desc
67897,B077C7395B,UK,GardenPrime 50 pack Garden Securing Pegs Anti-...,7.75,GardenPrime,NaN,50,NaN,"6""/150mm length, Galvanised (Rust Protected) S...",NaN,CHOOSE YOUR PERFECT PEG GardenPrime supply a w...
380339,B09D3TXRWY,UK,Amazon Brand - Mama Bear Fresh Lightly Fragran...,5.60,Amazon EU Sarl,NaN,56 Count (Pack of 6),5400606960435,NaN,NaN,6 x 56 wipes
7251,B09H4C9X6J,UK,OBVIS Cell Phone Pocket Self Adhesive Card Hol...,5.50,OBVIS,Black,NaN,NaN,Spandex,NaN,"3M adhesive backing stays strong, firm and lon..."
120077,B0857JXGRN,UK,Skullcandy Sesh Evo True Wireless Earbuds via ...,60.67,SKULLCANDY,Deep Red,NaN,S2TVW-N741,Plastic,NaN,24 hours total battery life and rapid charge: ...
410063,B002UPVVOM,UK,Set In Darkness,4.99,Orion,NaN,NaN,NaN,NaN,"Rankin, Ian",NaN


(1182181, 3)


,prev_items,next_item,locale
152563,['B092LYD1V8' 'B081R6P4BM' 'B08NXRRC76' 'B08BK...,B0B8RCN5DZ,UK
460453,['B083542MGJ' 'B093YG8964' 'B01KIQH2VU'],B0B8CBLW64,UK
674222,['B07KRN58WG' 'B010FJE9I4'],1509815872,UK
746121,['B08QNFYTVB' 'B08QNFYTVB'],B0001P05GS,UK
934391,['B006VCWJKA' 'B0045CGF0I' 'B09B4J5QLF'],B09TSRGR2Z,UK


In [11]:
products_df['author'].notnull().sum()

np.int64(18896)

In [ ]:
products_df['price'].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.99])

count     500180.00
mean     2021694.30
std      8762408.00
min            0.00
1%             1.99
5%             3.49
25%            7.99
50%           12.99
75%           23.99
80%           27.99
90%           59.99
95%     40000000.07
99%     40000000.07
max     40000000.07
Name: price, dtype: float64

In [21]:
products_df = products_df[(products_df['price'] < 10000) & (products_df['price'] > 0)].reset_index(drop=True)
products_df['price'].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.99])

count   473518.00
mean        23.34
std         64.51
min          0.01
1%           1.99
5%           3.49
25%          7.69
50%         12.98
75%         20.99
80%         24.95
90%         37.45
95%         62.00
99%        215.94
max       7999.00
Name: price, dtype: float64

In [20]:
products_df[products_df['author'].notnull()].sample(5)

,id,locale,title,price,brand,color,size,model,material,author,desc
58421,B0B86HB11T,UK,A Crown of Cursed Love,3.99,NaN,NaN,NaN,NaN,NaN,"Robinson, J L",NaN
143726,1801054843,UK,"Crowns and Gowns - Princesses, Mess Free Activ...",8.67,Imagine That Publishing,NaN,NaN,NaN,NaN,"McLean, Rachael",NaN
179205,0722538073,UK,How to Talk to Anyone: 92 Little Tricks For Bi...,6.68,HarperCollins Publishers,NaN,NaN,2724322333384,NaN,"Lowndes, Leil",NaN
324852,1471197190,UK,A Prequel to Practical Magic,7.48,Scribner UK,NaN,NaN,NaN,NaN,"Hoffman, Alice",NaN
241153,B07NP1S1JP,UK,Free Solo,7.99,NaN,NaN,NaN,NaN,NaN,Alex Honnold,NaN


In [27]:
# create price bins with data-driven cutoffs
# make at least 100 bins
num_bins = 100
price_bins = np.percentile(products_df['price'], np.linspace(0, 100, num_bins + 1))
products_df['price_bin'] = np.digitize(products_df['price'], price_bins, right=True)
products_df

,id,locale,title,price,brand,color,size,model,material,author,desc,price_bin
0,B087LZNPHS,UK,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid...",24.99,SOCHOW,Teal Green,127cm×150cm,NaN,100% Polyester,NaN,COLOR: The sherpa throw blanket is available i...,81
1,B08THFN1KX,UK,Hippowarehouse Personalised Photo Printed Mous...,9.95,Hippowarehouse,White,240mm x 190mm x 60mm,50245-Mat-Perso,Rubber,NaN,Competitively priced,37
2,0804185328,UK,"500 Easy Recipes for Every Machine, Both Stove...",16.49,Clarkson Potter,White,NaN,NaN,NaN,"Scarbrough, Mark",NaN,65
3,B09VBKDBW6,UK,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S...",11.99,TYHJOY,Black,NaN,FBA-sealer-black,Acrylonitrile Butadiene Styrene,NaN,【AFTER-SALE】This handheld food heat sealer sho...,46
4,B096ZW8B49,UK,Lucosobie Steering Wheel Lock - Car Anti-Theft...,26.99,Lucosobie,Black,NaN,NaN,Alloy Steel,NaN,🔐【 Anti-Friction & Customer First】Each box of ...,83
...,...,...,...,...,...,...,...,...,...,...,...,...
473513,B08D7KW8VK,UK,TOMHOUSEE Anime Cosplay Short Straight Hair Wi...,9.99,TOMHOUSEE,Deep Grey Yuki,NaN,NaN,Synthetic,NaN,NaN,37
473514,B073WXLXR9,UK,Crystals NEW brilliant ink twister bingo dabbe...,8.99,CRYSTALS,"Orange,blue,green,pink,red,purple",NaN,NaN,Plastic,NaN,NaN,32
473515,1529393833,UK,"Before I Do: the new, funny and unexpected lov...",4.50,Hodder Paperbacks,NaN,NaN,NaN,NaN,"Cousens, Sophie",NaN,10
473516,B0B3TJ1NDN,UK,"Black iPhone Charger Cable, iPhone Charger Bra...",4.49,AA-TECH,Black,2M,brd-ip-black-2022,Nylon Braided,NaN,Added Protection: An additional layer of prote...,10


In [36]:
le = LabelEncoder()
products_df['emb_author'] = 'author_' + products_df['author'].fillna('missing')
products_df['emb_price_bin'] = 'price_bin_' + products_df['price_bin'].astype(str)

le.fit(products_df['emb_author'].tolist() + products_df['emb_price_bin'].tolist())

products_df['emb_author'] = le.transform(products_df['emb_author'])
products_df['emb_price_bin'] = le.transform(products_df['emb_price_bin'])
products_df

,id,locale,title,price,brand,color,size,model,material,author,desc,price_bin,emb_author,emb_price_bin
0,B087LZNPHS,UK,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid...",24.99,SOCHOW,Teal Green,127cm×150cm,NaN,100% Polyester,NaN,COLOR: The sherpa throw blanket is available i...,81,8032,8111
1,B08THFN1KX,UK,Hippowarehouse Personalised Photo Printed Mous...,9.95,Hippowarehouse,White,240mm x 190mm x 60mm,50245-Mat-Perso,Rubber,NaN,Competitively priced,37,8032,8075
2,0804185328,UK,"500 Easy Recipes for Every Machine, Both Stove...",16.49,Clarkson Potter,White,NaN,NaN,NaN,"Scarbrough, Mark",NaN,65,6950,8096
3,B09VBKDBW6,UK,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S...",11.99,TYHJOY,Black,NaN,FBA-sealer-black,Acrylonitrile Butadiene Styrene,NaN,【AFTER-SALE】This handheld food heat sealer sho...,46,8032,8081
4,B096ZW8B49,UK,Lucosobie Steering Wheel Lock - Car Anti-Theft...,26.99,Lucosobie,Black,NaN,NaN,Alloy Steel,NaN,🔐【 Anti-Friction & Customer First】Each box of ...,83,8032,8113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473513,B08D7KW8VK,UK,TOMHOUSEE Anime Cosplay Short Straight Hair Wi...,9.99,TOMHOUSEE,Deep Grey Yuki,NaN,NaN,Synthetic,NaN,NaN,37,8032,8075
473514,B073WXLXR9,UK,Crystals NEW brilliant ink twister bingo dabbe...,8.99,CRYSTALS,"Orange,blue,green,pink,red,purple",NaN,NaN,Plastic,NaN,NaN,32,8032,8072
473515,1529393833,UK,"Before I Do: the new, funny and unexpected lov...",4.50,Hodder Paperbacks,NaN,NaN,NaN,NaN,"Cousens, Sophie",NaN,10,2494,8053
473516,B0B3TJ1NDN,UK,"Black iPhone Charger Cable, iPhone Charger Bra...",4.49,AA-TECH,Black,2M,brd-ip-black-2022,Nylon Braided,NaN,Added Protection: An additional layer of prote...,10,8032,8053


In [42]:
metadata = {
    'price_bins': price_bins.tolist(),
    'label_encoder': le.classes_.tolist()
}

In [ ]:
columns = ['brand', 'color', 'desc', 'material', 'model', 'size', 'title']

for column in columns:
    file_name = fr'..\data\amazon_kdd\products_{column}_emb.parquet'
    temp_df = pd.read_parquet(file_name)
    print(column)
    print(temp_df.shape)
    temp_df = temp_df.drop_duplicates(subset=[column], ignore_index=True)
    print(temp_df.shape)
    temp_df.to_parquet(file_name, index=False)
    print()

brand
(500180, 3)
(76349, 3)

color
(500180, 3)
(72069, 3)

desc
(500180, 3)
(297179, 3)

material
(500180, 3)
(14776, 3)

model
(500180, 3)
(202106, 3)

size
(500180, 3)
(74021, 3)

title
(500180, 3)
(482331, 3)

